In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
#define the WikipediaAPI which fetch our result with 200 characters
wiki_api = WikipediaAPIWrapper(top_k_results = 1, doc_content_chars_max=200)
#Make it as tool then our LLM can directly use it whenever it need
wiki_tool = WikipediaQueryRun(api_wrapper = wiki_api,
                              name = "wikipedia",
                              description="Search Wikipedia for factual information about general topics, such as countries, historical events, or public figures.")

In [3]:
wiki_tool

WikipediaQueryRun(description='Search Wikipedia for factual information about general topics, such as countries, historical events, or public figures.', api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Jasweer\\my_project\\Langchain\\Lvenv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200))

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(web_path = "https://docs.smith.langchain.com/")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
documents = text_splitter.split_documents(docs)
db = FAISS.from_documents(documents, OllamaEmbeddings(model = "llama2"))
retreiver = db.as_retriever()


USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\jaswe\AppData\Local\Temp\ipykernel_21264\2382940143.py:10: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  db = FAISS.from_documents(documents, OllamaEmbeddings(model = "llama2"))


In [5]:
from langchain.tools.retriever import create_retriever_tool
retrieval_tool = create_retriever_tool(retreiver, "langsmith_search", 
"Search for information about LangSmith. For any questions about LangSmith, you must use this tool")

In [6]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv_api = ArxivAPIWrapper(top_k_results = 1, doc_content_chars_max = 200)
arxiv_tool = ArxivQueryRun(api_wrapper = arxiv_api,
                           name="arxiv",
                            description="Search Arxiv for academic papers and research-related information.")
arxiv_tool.name

'arxiv'

In [7]:
tools = [wiki_tool, arxiv_tool, retrieval_tool]

In [8]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model = "llama2", temperature=0, format = "json")

In [9]:
from langchain import hub
prompt = hub.pull("hwchase17/structured-chat-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], input_types={}, partial_variables={}, template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\nBegin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. R

In [10]:
from langchain.agents import create_structured_chat_agent
agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt)

In [11]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, 
                               tools=tools, 
                               verbose=True,
                               handle_parsing_errors=True,
                               max_iterations=5)

In [13]:
response = agent_executor.invoke({"input": "What is OpenAI"})
print(response)



> Entering new AgentExecutor chain...
{
  "action": "wikipedia",
  "action_input": {"query": "OpenAI"}}Page: OpenAI
Summary: OpenAI, Inc. is an American artificial intelligence (AI) organization headquartered in San Francisco, California. It aims to develop "safe and beneficial" artificial general inte{
  "action": "arxiv",
  "action_input": {"query": "OpenAI"}}Published: 2016-06-05
Title: OpenAI Gym
Authors: Greg Brockman, Vicki Cheung, Ludwig Pettersson, Jonas Schneider, John Schulman, Jie Tang, Wojciech Zaremba
Summary: OpenAI Gym is a toolkit for reinfor{
  "action": "langsmith_search",
  "action_input": {"query": "OpenAI"}}Get started with LangSmith - Docs by LangChainDocs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationOverviewQuickstartsTrace an applicationEvaluate an applicationTest promptsAPI & SDKsAPI referencePython SDKJS/TS SDKPricingPlansPricing FAQDocs by LangChain home pagePythonSearch...⌘KAsk AIGitHubForumForumSearch...Navi

In [14]:
response["output"]

"OpenAI is an American artificial intelligence organization headquartered in San Francisco, California. It aims to develop 'safe and beneficial' artificial general intelligence."